In [5]:
from pydicom import dcmread
import os
import pandas as pd
import numpy as np

In [14]:
print(dcmread("/Users/simongutwein/Studium/Masterarbeit/DATA/ht4/Schulz^Per^^^0_strctr.dcm"))
print(dcmread("/Users/simongutwein/Studium/Masterarbeit/DATA/ht4/Schulz^Per^^^0_plan.dcm"))
print(dcmread("/Users/simongutwein/Studium/Masterarbeit/DATA/ht4/Schulz^Per^^^0_dose.dcm"))


Dataset.file_meta -------------------------------
(0002, 0000) File Meta Information Group Length  UL: 204
(0002, 0001) File Meta Information Version       OB: b'\x00\x01'
(0002, 0002) Media Storage SOP Class UID         UI: RT Structure Set Storage
(0002, 0003) Media Storage SOP Instance UID      UI: 1.2.840.10008.5.1.4.1.1.481.3.1634741912
(0002, 0010) Transfer Syntax UID                 UI: Explicit VR Little Endian
(0002, 0012) Implementation Class UID            UI: 1.2.276.0.7230010.3.0.3.6.2
(0002, 0013) Implementation Version Name         SH: 'OFFIS_DCMTK_362'
(0002, 0016) Source Application Entity Title     AE: 'CMS_SCU'
-------------------------------------------------
(0008, 0012) Instance Creation Date              DA: '20211020'
(0008, 0013) Instance Creation Time              TM: '165836.000000'
(0008, 0014) Instance Creator UID                UI: 2.16.840.1.114337
(0008, 0016) SOP Class UID                       UI: RT Structure Set Storage
(0008, 0018) SOP Instance UID 

In [7]:
# tests = [x for x in os.listdir("/Users/simongutwein/Studium/Masterarbeit/DATA") if not x.startswith(".")]

# for test in tests:
#     folder = os.path.join("/Users/simongutwein/Studium/Masterarbeit/DATA", test)
#     plan = [x for x in os.listdir(folder) if "plan" in x]
#     dat = dcmread(os.path.join(folder,plan[0]))
#     print(plan)
#     print(dat)
#     print("\n\n\n\n\n")



In [8]:
path = "/Users/simongutwein/Studium/Masterarbeit/DATA/Incoming"
files = [x for x in os.listdir(path) if not x.startswith(".")]

# files_temp = [x for x in files if "RP" in x or "RS" in x or "RD" in x]
# files_temp

In [9]:
# for file in files_temp:
#     dat = dcmread(os.path.join(path, file))
#     print(dat.PatientName)

In [10]:
info = []
for file in files:
    filepath = os.path.join(path, file)
    dat = dcmread(filepath, force=True)
    if hasattr(dat, "SliceLocation"):

        info.append({
            "filepath": filepath,
            "name": "".join(dat.PatientName),
            "filetype": file.split(".")[0],
            "slice_location": dat.SliceLocation,
            "SOP": None,
            "ref_struc": None,
            "ref_plan": None,
            "machine": None
        })

    else:  
        if hasattr(dat, "ReferencedRTPlanSequence"):
            ref_plan = dat.ReferencedRTPlanSequence[0].ReferencedSOPInstanceUID
        else:
            ref_plan = None

        if hasattr(dat, "ReferencedStructureSetSequence"):
            ref_struc = dat.ReferencedStructureSetSequence[0].ReferencedSOPInstanceUID
        else:
            ref_struc = None

        if hasattr(dat, "BeamSequence"):
            machine = dat.BeamSequence[0].TreatmentMachineName
        else:
            machine=None

        info.append({
            "filepath": filepath,
            "name": "".join(dat.PatientName),
            "filetype": file.split(".")[0],
            "slice_location": np.nan,
            "SOP": dat.SOPInstanceUID,
            "ref_struc": ref_struc, 
            "ref_plan": ref_plan,
            "machine": machine
        })


In [11]:
df = pd.DataFrame(info)
print(df)

                                               filepath  \
0     /Users/simongutwein/Studium/Masterarbeit/DATA/...   
1     /Users/simongutwein/Studium/Masterarbeit/DATA/...   
2     /Users/simongutwein/Studium/Masterarbeit/DATA/...   
3     /Users/simongutwein/Studium/Masterarbeit/DATA/...   
4     /Users/simongutwein/Studium/Masterarbeit/DATA/...   
...                                                 ...   
2661  /Users/simongutwein/Studium/Masterarbeit/DATA/...   
2662  /Users/simongutwein/Studium/Masterarbeit/DATA/...   
2663  /Users/simongutwein/Studium/Masterarbeit/DATA/...   
2664  /Users/simongutwein/Studium/Masterarbeit/DATA/...   
2665  /Users/simongutwein/Studium/Masterarbeit/DATA/...   

                        name filetype  slice_location   SOP ref_struc  \
0              Wagner^Udo^^^       CT         -283.44  None      None   
1      Sickinger^Fridolin^^^       CT         -300.97  None      None   
2     Bierfreund^Siegmund^^^       CT         -157.10  None      None   

In [12]:
patients = df["name"].unique()

In [13]:
writer = pd.ExcelWriter("/Users/simongutwein/Studium/Masterarbeit/plans_1.xlsx")
for patient in patients:
    temp_df = df[df["name"] == patient]
    temp_df = temp_df.sort_values(by=["slice_location"])
    temp_df.to_excel(writer, sheet_name=patient)
    add_files = temp_df[temp_df["slice_location"].isnull()]
    add_files.to_excel(writer, sheet_name=patient)

    ct_files = temp_df[temp_df["slice_location"].notnull()]
    for num, file in enumerate(ct_files.iterrows()):
        os.rename(file[1][0], f"{path}/{patient}CT_image{num}.dcm")
    
    #print(add_files)

    dose_files = add_files[add_files["filetype"] == "RD"]
    plan_files = add_files[add_files["filetype"] == "RP"]
    struc_files = add_files[add_files["filetype"] == "RS"]
    for num, dose_file in enumerate(dose_files.iterrows()):
        dose_sop = dose_file[1][4]
        plan_sop = dose_file[1][6]
        if plan_sop in plan_files["SOP"].unique():
            plan_file = plan_files[plan_files["SOP"]==plan_sop]
            machine = plan_file["machine"].iloc[0]
            print(machine)
            print("mr" in machine.lower())
            if "mr" in machine.lower():
                struc_sop = plan_file["ref_struc"].iloc[0]
                if struc_sop in struc_files["SOP"].unique():
                    RS = add_files[add_files["SOP"] == struc_sop]["filepath"].iloc[0]
                    RP = add_files[add_files["SOP"] == plan_sop]["filepath"].iloc[0]
                    RD = add_files[add_files["SOP"] == dose_sop]["filepath"].iloc[0]
                    #print(patient, RS, RP, RD)
                    os.rename(RS, f"{path}/{patient}{num}_strctr.dcm")
                    os.rename(RP, f"{path}/{patient}{num}_plan.dcm")
                    os.rename(RD, f"{path}/{patient}{num}_dose.dcm")


writer.save()


MRL
True
Linac-ROT
False
MR-Linac
True
MR-Linac
True
MR-Linac
True
MR-Linac
True
MR-Linac
True
MR-Linac
True
MR-Linac
True
MRL
True
Linac-Rot
False
Linac-Rot
False
MRL
True
Linac-Rot
False
MR-Linac
True
Linac-Rot
False
MR-Linac
True
MR-Linac
True
MR-Linac
True
MR-Linac
True
MR-Linac
True
MR-Linac
True
Linac-Rot
False
MR-Linac
True
MR-Linac
True
MR-Linac
True
MRL
True
MR-Linac
True
